Using code from Signatrix: https://github.com/signatrix/efficientdet

In [ ]:
!git clone https://github.com/signatrix/efficientdet.git
%cd efficientdet/ 
!pip install -r requirements.txt

#import some useful modules
import pandas as pd
import numpy as np
from IPython.display import Image, clear_output  # to display images
#from utils.google_utils import gdrive_download  # to download models/datasets


Cloning into 'efficientdet'...
remote: Enumerating objects: 144, done.
remote: Total 144 (delta 0), reused 0 (delta 0), pack-reused 144
Receiving objects: 100% (144/144), 28.03 MiB | 35.30 MiB/s, done.
Resolving deltas: 100% (68/68), done.
/content/efficientdet/efficientdet/efficientdet


requirments.txt has the following modules: 
efficientnet_pytorch
tensorboardX
pycocotools

In [1]:
#mount at google drive
from google.colab import drive
drive.mount('/content/drive')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelbase.py", line 729, in _input_request
    ident, reply = self.session.recv(self.stdin_socket, 0)
  File "/usr/local/lib/python3.6/dist-packages/jupyter_client/session.py", line 803, in recv
    msg_list = socket.recv_multipart(mode, copy=copy)
  File "/usr/local/lib/python3.6/dist-packages/zmq/sugar/socket.py", line 491, in recv_multipart
    parts = [self.recv(flags, copy=copy, track=track)]
  File "zmq/backend/cython/socket.pyx", line 791, in zmq.backend.cython.socket.Socket.recv
  File "zmq/backend/cython/socket.pyx", line 827, in zmq.backend.cython.socket.Socket.recv
  File "zmq/backend/cython/socket.pyx", line 186, in zmq.backend.cython.socket._recv_copy
  File "zmq/backend/cython/checkrc.pxd", line 13, in zmq.backend.cython.checkrc._check_rc
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/li

TypeError: ignored

In [ ]:
#create dataframe 'df' to reformat training images for effifcientdet
import tqdm
df = pd.read_csv('/content/drive/My Drive/Copy of wheat/train.csv')
bboxs = np.stack(df['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
for i, column in enumerate(['x', 'y', 'w', 'h']):
    df[column] = bboxs[:,i]
df.drop(columns=['bbox'], inplace=True)
df['x_center'] = df['x'] + df['w']/2
df['y_center'] = df['y'] + df['h']/2
df['classes'] = 0
from tqdm.auto import tqdm
import shutil as sh
df = df[['image_id','x', 'y', 'w', 'h','x_center','y_center','classes']]


In [ ]:
!git clone https://github.com/NVIDIA/apex && cd apex && pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" . --user && cd .. && rm -rf apex

Cloning into 'apex'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 7367 (delta 3), reused 0 (delta 0), pack-reused 7353
Receiving objects: 100% (7367/7367), 13.89 MiB | 23.74 MiB/s, done.
Resolving deltas: 100% (4974/4974), done.
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-puj8l9_8
Created temporary directory: /tmp/pip-req-tracker-66i0nhrh
Created requirements tracker '/tmp/pip-req-tracker-66i0nhrh'
Created temporary directory: /tmp/pip-install-lm6h1nlx
Processing /content/efficientdet/apex
  Created temporary directory: /tmp/pip-req-build-r6ol_8qf
  Added file:///content/efficientdet/apex to build tracker '/tmp/pip-req-tracker-66i0nh

In [ ]:
import os 
index = list(set(df.image_id))
source = 'train'
if True:
    for fold in [0]:
        val_index = index[len(index)*fold//5:len(index)*(fold+1)//5]
        for name,mini in tqdm(df.groupby('image_id')):
            if name in val_index:
                path2save = 'val2017/'
            else:
                path2save = 'train2017/'
            if not os.path.exists('/content/wheat1/fold{}/labels/'.format(fold)+path2save):
                os.makedirs('/content/wheat1/fold{}/labels/'.format(fold)+path2save)
            with open('/content/wheat1/fold{}/labels/'.format(fold)+path2save+name+".txt", 'w+') as f:
                row = mini[['classes','x_center','y_center','w','h']].astype(float).values
                row = row/1024
                row = row.astype(str) 
                for j in range(len(row)):
                    text = ' '.join(row[j])
                    f.write(text)
                    f.write("\n")
            if not os.path.exists('/content/wheat1/fold{}/images/{}'.format(fold,path2save)):
                os.makedirs('/content/wheat1/fold{}/images/{}'.format(fold,path2save))
            sh.copy("/content/drive/My Drive/Copy of wheat/{}/{}.jpg".format(source,name),'/content/wheat1/fold{}/images/{}/{}.jpg'.format(fold,path2save,name))

In [ ]:
#run train.py 
!python train.py --image_size 1024 --batch_size 2 --num_epochs 50 --data_path '/content/wheat1/'

loading annotations into memory...
Traceback (most recent call last):
  File "train.py", line 189, in <module>
    train(opt)
  File "train.py", line 58, in train
    transform=transforms.Compose([Normalizer(), Augmenter(), Resizer()]))
  File "/content/efficientdet/efficientdet/efficientdet/src/dataset.py", line 17, in __init__
    self.coco = COCO(os.path.join(self.root_dir, 'annotations', 'instances_' + self.set_name + '.json'))
  File "/usr/local/lib/python3.6/dist-packages/pycocotools/coco.py", line 84, in __init__
    dataset = json.load(open(annotation_file, 'r'))
FileNotFoundError: [Errno 2] No such file or directory: '/content/wheat1/annotations/instances_train2017.json'
